In [6]:
import os
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt, welch
from scipy.ndimage import uniform_filter1d
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams["text.usetex"] = False

# === Parameters ===
input_folder = "Raw Input"    # <-- change this
output_folder = "Peakfinder Input"
psd_folder = "[0] PSD_plots"
export_folder = r"C:\Users\Galahad\Desktop\Thesis\images"
os.makedirs(output_folder, exist_ok=True)
os.makedirs(psd_folder, exist_ok=True)

# Butterworth filter settings (narrower band)
bp_lowcut = 1.0
bp_highcut = 120.0
notch_low = 40.0
notch_high = 55.0
notch_order = 1
order = 4

# Welch PSD settings
nperseg = 1024
noverlap = nperseg // 2


first_psd_saved = False

# --- Filter functions ---
def butter_bandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandstop(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='bandstop')
    return b, a

def apply_filters(signal, fs):
    # Bandpass 5-60 Hz
    b_bp, a_bp = butter_bandpass(bp_lowcut, bp_highcut, fs, order)
    filtered = filtfilt(b_bp, a_bp, signal)
    # Notch 49-51 Hz
    b_notch, a_notch = butter_bandstop(notch_low, notch_high, fs, notch_order)
    filtered = filtfilt(b_notch, a_notch, filtered)
    # Smoothing (moving average, 5 ms window)
    window_size = max(1, int(fs * 0.005))
    filtered = uniform_filter1d(filtered, size=window_size)
    return filtered

# --- Helper function ---
def find_header_row(filepath):
    """Find row where 'MS' and 'ValueG' appear"""
    with open(filepath, "r", encoding="utf-8", errors="ignore") as f:
        for i, line in enumerate(f):
            if "MS" in line and "ValueG" in line:
                return i
    return 0

# --- Process files ---
files = [f for f in os.listdir(input_folder) if f.lower().endswith(".csv")]
print(f"Found {len(files)} CSV files.")

# ...existing code...

# ...existing code...

for filename in files:
    filepath = os.path.join(input_folder, filename)
    print(f"\nProcessing {filepath} ...")

    # Load CSV, skip metadata
    header_row = find_header_row(filepath)
    df = pd.read_csv(filepath, skiprows=header_row)

    if "MS" not in df.columns or "ValueG" not in df.columns:
        print("⚠️ Missing 'MS' or 'ValueG'. Skipping.")
        continue

    # Extract time and signal
    time = df["MS"].values
    signal = df["ValueG"].values

    # Convert ms → s if needed
    if np.max(time) > 1000:
        time = time / 1000.0

    # Compute sampling frequency robustly
    dt_values = np.diff(time)
    dt_clean = dt_values[dt_values > 0]  # ignore resets
    dt = np.median(dt_clean)
    fs = 1.0 / dt
    print(f"  Sampling rate ≈ {fs:.2f} Hz")

    # --- Apply bandpass + notch filter + smoothing ---
    filtered_signal = apply_filters(signal, fs)
    df["ValueG"] = filtered_signal  # <-- Overwrite ValueG column

    # --- Save filtered CSV ---
    out_csv_path = os.path.join(output_folder, filename)
    df.to_csv(out_csv_path, index=False)
    print(f"  ✅ Saved filtered CSV: {out_csv_path}")

    # --- Compute Welch PSD for both raw and filtered ---
    f_raw, Pxx_raw = welch(signal, fs=fs, nperseg=nperseg, noverlap=noverlap, window='hann')
    f_filt, Pxx_filt = welch(filtered_signal, fs=fs, nperseg=nperseg, noverlap=noverlap, window='hann')

    # --- Plot PSD comparison ---
    plt.figure(figsize=(8,4))
    plt.semilogy(f_raw, Pxx_raw, label="Raw")
    plt.semilogy(f_filt, Pxx_filt, label="Filtered")
    plt.xlim([0, 150])
    plt.xlabel("Frequency (Hz)")
    plt.ylabel(r"PSD (V$^2$/Hz)")
    plt.legend()
    try:
        plt.tight_layout()
    except Exception:
        pass
    # Save PSD plot only for the first file
    if not first_psd_saved:
        base_name = os.path.splitext(filename)[0]
        psd_png = os.path.join(psd_folder, base_name + "_PSD.png")
        psd_pgf = os.path.join(export_folder, "PSD.pgf")

        # Configure PGF: do NOT call external LaTeX for text when writing .pgf
        # (use Matplotlib's mathtext renderer instead)
        mpl.rcParams.update({
            "pgf.texsystem": "pdflatex",
            "font.family": "serif",
            "text.usetex": False,   # important: avoid injecting raw TeX into .pgf
            "pgf.rcfonts": False,
        })

        plt.savefig(psd_png)
        try:
            # explicitly request PGF format
            plt.savefig(psd_pgf, format="pgf")
            print(f"  ✅ Saved PSD PNG and PGF: {psd_png}, {psd_pgf}")
        except Exception as e:
            print(f"  ⚠️ PGF save failed ({e}). PNG saved: {psd_png}")
        first_psd_saved = True
    else:
        print("  ⏭ Skipping PSD save for this file")
    plt.close()

Found 18 CSV files.

Processing Raw Input\Animal10_April16.CSV ...
  Sampling rate ≈ 500.00 Hz
  ✅ Saved filtered CSV: Peakfinder Input\Animal10_April16.CSV
  ✅ Saved PSD PNG and PGF: [0] PSD_plots\Animal10_April16_PSD.png, C:\Users\Galahad\Desktop\Thesis\images\PSD.pgf

Processing Raw Input\Animal10_June26.CSV ...
  Sampling rate ≈ 500.00 Hz
  ✅ Saved filtered CSV: Peakfinder Input\Animal10_June26.CSV
  ⏭ Skipping PSD save for this file

Processing Raw Input\Animal10_May20.CSV ...
  Sampling rate ≈ 500.00 Hz
  ✅ Saved filtered CSV: Peakfinder Input\Animal10_May20.CSV
  ⏭ Skipping PSD save for this file

Processing Raw Input\Animal15_April16.CSV ...
  Sampling rate ≈ 500.00 Hz
  ✅ Saved filtered CSV: Peakfinder Input\Animal15_April16.CSV
  ⏭ Skipping PSD save for this file

Processing Raw Input\Animal15_June26.CSV ...
  Sampling rate ≈ 500.00 Hz
  ✅ Saved filtered CSV: Peakfinder Input\Animal15_June26.CSV
  ⏭ Skipping PSD save for this file

Processing Raw Input\Animal15_May20.CSV ...


In [11]:
# ...existing code...
import os
import matplotlib as mpl
import matplotlib.pyplot as plt

# export folder (change if needed)
export_folder = r"C:\Users\Galahad\Desktop\Thesis\images"
export_folder_png = r"C:\Users\Galahad\Desktop\Thesis\images\png"
os.makedirs(export_folder, exist_ok=True)
os.makedirs(export_folder_png, exist_ok=True)

# Configure PGF (requires a LaTeX engine like pdflatex on your system)
mpl.rcParams.update({
    "pgf.texsystem": "pdflatex",
    "font.family": "serif",
    "text.usetex": False,
    "pgf.rcfonts": False,
})

# Define time window (ms)
start_time_ms = 10000
end_time_ms = 11000
time_ms = time * 1000
indices = np.where((time_ms >= start_time_ms) & (time_ms <= end_time_ms))

plt.figure(figsize=(8,4))
plt.plot(time_ms[indices], signal[indices], label="Raw", rasterized=True)
plt.plot(time_ms[indices], filtered_signal[indices], label="Filtered", alpha=0.8, rasterized=True)
plt.xlabel("Time (ms)")
plt.ylabel("ValueG")
plt.legend()

pgf_path = os.path.join(export_folder, "trace.pgf")
png_path = os.path.join(export_folder_png, "trace.png")
pdf_path = os.path.join(export_folder, "trace.pdf")

try:
    plt.savefig(pgf_path, format="pgf")
    print(f"Saved PGF: {pgf_path}")
except Exception as e:
    print(f"PGF save failed ({e}), saving PDF instead.")
    plt.savefig(pdf_path)
plt.savefig(png_path, dpi=150)
plt.close()
print(f"Saved PNG: {png_path}")
# ...existing code...

Saved PGF: C:\Users\Galahad\Desktop\Thesis\images\trace.pgf
Saved PNG: C:\Users\Galahad\Desktop\Thesis\images\png\trace.png
